In [10]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 경고 뜨지 않게...
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['font.size'] = 16
plt.rcParams['figure.figsize'] = 20, 10
plt.rcParams['axes.unicode_minus'] = False

# 랜덤 모듈
import random

# 학습 모델 저장 및 복원
import pickle

# 딥러닝 라이브러리
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation

# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 랜덤시드 설정
# 데이터를 랜덤하게 섞거나 가중치를 랜덤하게 설정하는 등..
# 작업에서 랜덤을 적용하는 경우가 더러 있다.
# 이에, 시드를 고정시킨다.
random_seed = 1
np.random.seed(random_seed)
random.seed(random_seed)
tf.random.set_seed(random_seed)

# 현재 프로젝트에서 GPU 메모리 사용을 필요한 만큼만 쓸 수 있도록 한다.
# 컴퓨터에 있는 GPU 정보들을 가져온다.

gpus = tf.config.experimental.list_physical_devices('GPU')
# gpu가 있다면...
if len(gpus) > 0 :
    try :
        for gpu in gpus :
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e :
        print(e)

In [11]:
# 데이터를 불러온다.
df1 = pd.read_csv('./data/ThoraricSurgery.csv', header = None)
df1

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,293,1,3.80,2.80,0,0,0,0,0,0,12,0,0,0,1,0,62,0
1,1,2,2.88,2.16,1,0,0,0,1,1,14,0,0,0,1,0,60,0
2,8,2,3.19,2.50,1,0,0,0,1,0,11,0,0,1,1,0,66,1
3,14,2,3.98,3.06,2,0,0,0,1,1,14,0,0,0,1,0,80,1
4,17,2,2.21,1.88,0,0,1,0,0,0,12,0,0,0,1,0,56,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465,98,6,3.04,2.40,2,0,0,0,1,0,11,0,0,0,1,0,76,0
466,369,6,3.88,2.72,1,0,0,0,1,0,12,0,0,0,1,0,77,0
467,406,6,5.36,3.96,1,0,0,0,1,0,12,0,0,0,0,0,62,0
468,25,8,4.32,3.20,0,0,0,0,0,0,11,0,0,0,0,0,58,1


In [12]:
# 입력과 결과로 나눈다.
X = df1.drop(17, axis=1)
y = df1[17]

display(X)
display(y)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,293,1,3.80,2.80,0,0,0,0,0,0,12,0,0,0,1,0,62
1,1,2,2.88,2.16,1,0,0,0,1,1,14,0,0,0,1,0,60
2,8,2,3.19,2.50,1,0,0,0,1,0,11,0,0,1,1,0,66
3,14,2,3.98,3.06,2,0,0,0,1,1,14,0,0,0,1,0,80
4,17,2,2.21,1.88,0,0,1,0,0,0,12,0,0,0,1,0,56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465,98,6,3.04,2.40,2,0,0,0,1,0,11,0,0,0,1,0,76
466,369,6,3.88,2.72,1,0,0,0,1,0,12,0,0,0,1,0,77
467,406,6,5.36,3.96,1,0,0,0,1,0,12,0,0,0,0,0,62
468,25,8,4.32,3.20,0,0,0,0,0,0,11,0,0,0,0,0,58


0      0
1      0
2      1
3      1
4      0
      ..
465    0
466    0
467    0
468    1
469    0
Name: 17, Length: 470, dtype: int64

In [13]:
# 신경망을 설계한다.
# 신경망 모델을 관리하는 객체를 생성한다
# 여기에 층을 쌓아준다.
model = Sequential()

# 층을 쌓는 부분
# 케라스는 은닉층과 출력층만 설정한다.
# 첫번째 은닉층을 설정할 때 input_dim을 통해 입력층의 노드 수를 설정한다.
# 입력층의 노드의 수는 입력데이터의 수와 일치해야 한다.
# 은닉층의 노드의 수는 정해진 규칙은 없다.
# 많은면 학습시간이 오래걸리지만 학습을 잘 하게 되고
# 너무 학습을 많이하게 되면 불필요하게 오랜 시간이 걸릴 수 있다.

# Dense : 레이어 내의 모든 노드들은 선형 회귀를 수행한다.
model.add(Dense(30, input_dim=17))
# 활성화 함수
# relu : 역전파 및 경사하강법 수행시 가중치 소실이 발생하지 않도록
# 예방하기 위한 활성화 함수
model.add(Activation('relu'))

# 마지막은 출력층에 해당한다.
# 출력층의 노드의 개수는 예측하고자 하는 값의 수와 일치해야 한다.
model.add(Dense(1))
# 활성화 함수
# 출력층의 활성화 함수는 최종 결과를 계산하기 위해 사용한다.
# 2진 분류 : sigmoid
# 다중 분류 : softmax
# 회귀 : 없음
model.add(Activation('sigmoid'))

In [14]:
# 신경망 모델을 컨파일
# loss : 학습시 수행하는 평가때 사용할 손실함수
# 2진분류 : binary_crossentropy
# 다중분류 : categorical_crossentropy
# 회귀 : mean_squared_error

# opimizer : loss에서 설정한 함수를 통해 구한 손실값을 기반으로
# 신경망 모델을 보정하는 함수
# adam

# metrics : 추가로 평가하고자 하는 평가 기준을 설정한다.
# 분류인 경우에만 사용하고 정도화를 추가적으로 보기위해 설정한다.
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 30)                540       
                                                                 
 activation_4 (Activation)   (None, 30)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 31        
                                                                 
 activation_5 (Activation)   (None, 1)                 0         
                                                                 
Total params: 571
Trainable params: 571
Non-trainable params: 0
_________________________________________________________________


In [20]:
# 학습
# eqcohs : 학습 횟수. 학습 횟수가 너무 많으면 과적합된다.
# 무조건 많이 주세요 ~~~ 나중에 중단 시킬 수 있습니다!!!!
# batch_size : 메모리에 할번에 올릴 데이터의 양(행의 개수)
# 적게 주면 메모리에 여유가 있지만 오래걸린다.
# 적당히... 무조건 많이 주시고 오류나면 낮추세요~
model.fit(X, y, epochs=30, batch_size=10)

Epoch 1/30
47/47 [==============================] - 0s 3ms/step - loss: 0.1409 - accuracy: 0.8511
Epoch 2/30
47/47 [==============================] - 0s 3ms/step - loss: 0.1430 - accuracy: 0.8553
Epoch 3/30
47/47 [==============================] - 0s 2ms/step - loss: 0.1413 - accuracy: 0.8511
Epoch 4/30
47/47 [==============================] - 0s 2ms/step - loss: 0.1421 - accuracy: 0.8489
Epoch 5/30
47/47 [==============================] - 0s 2ms/step - loss: 0.1398 - accuracy: 0.8511
Epoch 6/30
47/47 [==============================] - 0s 2ms/step - loss: 0.1419 - accuracy: 0.8511
Epoch 7/30
47/47 [==============================] - 0s 2ms/step - loss: 0.1412 - accuracy: 0.8468
Epoch 8/30
47/47 [==============================] - 0s 2ms/step - loss: 0.1384 - accuracy: 0.8553
Epoch 9/30
47/47 [==============================] - 0s 2ms/step - loss: 0.1405 - accuracy: 0.8511
Epoch 10/30
47/47 [==============================] - 0s 2ms/step - loss: 0.1424 - accuracy: 0.8511
Epoch 11/30
47/47 [

In [21]:
# 학습이 완료된 모델에 대해서 손실률과 정확도를 평가한다.
r1 = model.evaluate(X, y)
print(f'손실률 : {r1[0]}')
print(f'정확도 : {r1[1]}')

15/15 [==============================] - 0s 2ms/step - loss: 0.1368 - accuracy: 0.8596
손실률 : 0.13676761090755463
정확도 : 0.8595744967460632


In [29]:
# 예측한다.
pred1 = model.predict(X)
pred1

15/15 [==============================] - 0s 2ms/step


array([[1.14818247e-14],
       [4.24996078e-01],
       [6.10486567e-01],
       [4.27514136e-01],
       [2.73493916e-01],
       [1.11084588e-01],
       [6.18848391e-02],
       [3.54415268e-01],
       [1.87098578e-01],
       [1.37821507e-05],
       [4.10376106e-06],
       [7.70526603e-05],
       [4.89451622e-07],
       [5.77805531e-06],
       [4.30918945e-09],
       [2.24352476e-07],
       [3.62807384e-07],
       [5.41893428e-07],
       [1.79483806e-08],
       [1.20199717e-08],
       [1.38220839e-08],
       [7.75353026e-09],
       [5.13241094e-09],
       [6.56099508e-10],
       [3.59883345e-10],
       [1.25476754e-10],
       [3.28402694e-10],
       [2.66369340e-11],
       [4.81391420e-11],
       [1.82519017e-12],
       [1.91766240e-13],
       [6.97099306e-14],
       [2.36295612e-14],
       [3.88182028e-15],
       [1.83665923e-15],
       [8.25250067e-21],
       [2.06842415e-16],
       [9.81945510e-17],
       [9.65314084e-17],
       [2.27082704e-16],


In [26]:
# 0.5 초과는 1 아닌 것은 0으로 환산한다.
result1 = (pred1 > 0.5).astype(int)
result1 = result1.reshape(-1)
result1

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [35]:
# 정확도 확인
score1 = accuracy_score(y, result1)
score1

0.8595744680851064